In [26]:
import sys
import numpy as np
import pandas
from astropy import constants as const
from astropy import units as u
from astropy.io import fits
from astropy.io import ascii
import random
import matplotlib.pyplot as plt
import wget
import seaborn as sns
import random
import os
import warnings
warnings.filterwarnings("ignore")
#import healpy

In [57]:
hdulistBricksSummary = fits.open('../bricks_data/survey-bricks-dr9-south.fits')
dataSummary = hdulistBricksSummary[1].data
bricknameSummary = dataSummary.field('brickname')
brickidSummary = dataSummary.field('brickid')
brick_galaxy_info = np.array([[220271,34,10,177]])

[328928 327488 326048 ... 515285 516481 517675]


In [56]:
randomint = random.randint(0, len(bricknameSummary))
brickname = bricknameSummary[randomint] 
brickid = brickidSummary[randomint] 
print(brickname, brickid)

3451p027 347588


In [12]:
folder = brickid[:3]
url = f'https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/south/tractor/{folder}/tractor-{brickid}.fits'
wget.download(url, '../bricks_data/tractor/')

'../bricks_data/tractor//tractor-0056p090.fits'

In [22]:
brickid = '0374m195'
hdulistSingleBrick = fits.open(f'../bricks_data/tractor/tractor-{brickid}.fits')
data = hdulistSingleBrick[1].data


In [23]:
def is_LRG_target(g,r,z,W1):
    if not (18.01 < z < 20.41):
        return False
    if not (0.75 < (r - z) < 2.45):
        return False
    if not (-0.6 < (z - W1) - 0.8*(r - z)):
        return False
    if not ((z - 17.18)/2 < (r - z) < (z - 15.11)/2):
        return False
    if not (((r - z) > 1.15) or ((g-r) > 1.65)):
        return False
    return True
    
def is_ELG_target(g,r,z):
    if not (21.0 < g < 23.45):
        return False
    if not (0.3 < (r - z) < 1.6):
        return False
    if not ((g-r) < 1.15*(r-z) - 0.15):
        return False
    if not ((g-r) < 1.6 - 1.2*(r-z)):
        return False
    return True

def is_QSO_target(g,r,z,maskbit):
    if r > 22.7:
        return False
    if r < 17.5:
        return False
    if not g - r < 1.3:
        return False
    if not (-0.4 < r - z < 1.1):
        return False
    return True
        

In [28]:
#fluxes and magnitude
flux_g = data.field('flux_g')
flux_r = data.field('flux_r')
flux_z = data.field('flux_z')
flux_w1 = data.field('flux_w1')
flux_w2 = data.field('flux_w2')
flux_w3 = data.field('flux_w3')
flux_w4 = data.field('flux_w4')

mw_transmission_g = data.field('mw_transmission_g')
mw_transmission_r = data.field('mw_transmission_r')
mw_transmission_z = data.field('mw_transmission_z')
mw_transmission_w1 = data.field('mw_transmission_w1')
mw_transmission_w2 = data.field('mw_transmission_w2')
mw_transmission_w3 = data.field('mw_transmission_w3')
mw_transmission_w4  = data.field('mw_transmission_w4')

#correcting for extinction
flux_g_corrected = flux_g - mw_transmission_g
flux_r_corrected = flux_r - mw_transmission_r
flux_z_corrected = flux_z - mw_transmission_z
flux_w1_corrected = flux_w1 - mw_transmission_w1
flux_w2_corrected = flux_w2 - mw_transmission_w2
flux_w3_corrected = flux_w3 - mw_transmission_w3
flux_w4_corrected = flux_w4 - mw_transmission_w4



mag_g = 22.5-2.5*np.log10(flux_g_corrected)
mag_r = 22.5-2.5*np.log10(flux_r_corrected)
mag_z = 22.5-2.5*np.log10(flux_z_corrected)
mag_w1 = 22.5-2.5*np.log10(flux_w1_corrected)
mag_w2 = 22.5-2.5*np.log10(flux_w2_corrected)
mag_w3 = 22.5-2.5*np.log10(flux_w3_corrected)
mag_w4 = 22.5-2.5*np.log10(flux_w4_corrected)

#Retrieving the maskbits for quasar detection

maskbits = data.field('maskbits')

target_label_array = np.zeros(len(mag_g))
for i in range(len(mag_g)):
    if is_LRG_target(mag_g[i], mag_r[i], mag_z[i], mag_w1[i]):
        target_label_array[i] = 1
        continue
    if is_ELG_target(mag_g[i], mag_r[i], mag_z[i]):
        target_label_array[i] = 2
        continue
    if is_QSO_target(mag_g[i], mag_r[i], mag_z[i], maskbits[i]):
        target_label_array[i] = 3
        continue



In [53]:

lrg = (target_label_array == 1).sum()
elg = (target_label_array == 2).sum()
qso = (target_label_array == 3).sum()


brick_galaxy_info = np.append(brick_galaxy_info, brick_galaxy_info, axis=0)
print(brick_galaxy_info)

[[220271     34     10    177]
 [220271     34     10    177]]


In [14]:
os.remove(f'../bricks_data/tractor/tractor-{brickid}.fits')